<a href="https://colab.research.google.com/github/yzbaya/FAD.Machine-Learning/blob/main/FAD_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [24]:
data = pd.read_csv('histoFAD.csv', encoding='latin1')

<ipython-input-24-a38ad0390545>:1: DtypeWarning: Columns (5,14,22,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('histoFAD.csv', encoding='latin1')


In [25]:
numeric_features = ['NJR','CMJ']
print(data[numeric_features].dtypes)
#extract et imputer les colonnes numeriques
numeric_features = ['NJR','CMJ']
imputer_numeric=SimpleImputer(strategy='median')
data[numeric_features]=imputer_numeric.fit_transform(data[numeric_features])

NJR      int64
CMJ    float64
dtype: object


**Préparation des Données **

In [26]:
# List des colonnes à convertir et à imputer
columns_to_convert = ['N°','Vente 2024','Vente 2022','Vente 2023','Ecoulement']
# print('autres colonnes')
# print(data[columns_to_convert].dtypes)
def convert_data(data,columns_to_convert):
    for column in columns_to_convert:
        data[column] = data[column].replace({',': '.'}, regex=True)  # Remplacer les virgules par des points
        data[column] = pd.to_numeric(data[column], errors='coerce')  # Convertir en numérique
        imputer_cols = SimpleImputer(strategy='median')
        data[column] = imputer_cols.fit_transform(data[[column]])  # Imputer les valeurs manquantes
        # data[column] = pd.Series(data[column].flatten()).apply(lambda x: max(x, 0))  # Remplacer les valeurs négatives par zéro
    return data


data=convert_data(data,columns_to_convert)
#test
print('colonnes non num')
print(data[columns_to_convert].dtypes)

print("Vérification générale des valeurs manquantes:")
print(data[columns_to_convert].isna().sum())
print(data[numeric_features].isna().sum())

colonnes non num
N°            float64
Vente 2024    float64
Vente 2022    float64
Vente 2023    float64
Ecoulement    float64
dtype: object
Vérification générale des valeurs manquantes:
N°            0
Vente 2024    0
Vente 2022    0
Vente 2023    0
Ecoulement    0
dtype: int64
NJR    0
CMJ    0
dtype: int64


In [27]:
# Sélection et imputation des données catégorielles
categorical_features = ['Pareto']
imputer_mostfrequent = SimpleImputer(strategy='most_frequent')
data[categorical_features] = imputer_mostfrequent.fit_transform(data[categorical_features])
print(data[categorical_features].isna().sum())

data['Pareto_copy'] = data['Pareto']
# Mapping importance des pièces
pareto_importance = {
    'AA': 1, 'AB': 2, 'AC': 3, 'AD': 4,
    'BA': 5, 'BB': 6, 'BC': 7, 'BD': 8,
    'CA': 9, 'CB': 10, 'CC': 11, 'CD': 12,
    'DA': 13, 'DB': 14, 'DC': 15, 'DD': 16
}
data['Importance'] = data['Pareto_copy'].map(pareto_importance)

# Encodage one-hot
data = pd.get_dummies(data, columns=['Pareto_copy'], prefix='Pareto')


# Afficher les premières lignes du dataframe pour une vérification visuelle
print(data.head())


Pareto    0
dtype: int64
          N°                                        Désignation  \
0  1100936.0  JAUGE + POMPE DIESEL-A2124700194-MERCEDES S212...   
1     1090.0         PARAH.ROUE  068198171 11*85*105  AD 80-100   
2     1110.0                       SUPPORT BV -811399151A- AD80   
3     2113.0  BOUCHON CACHE SOUPAPE -06B103485C- GOLF 5-6/PO...   
4     2234.0               ROTULE DIRECTION -861419812- AUDI 80   

  Plage de Stock Enseigne Stocks Qté sur commande vente S-CVTE Prix unitaire  \
0            VIF     ACHR     1.                     0.   1.00       219.980   
1            VIF     FEBI    44.                     0.  44.00        24.830   
2           LENT     FEBI    21.                     0.  21.00        30.890   
3            VIF     FEBI    41.                     0.  41.00         6.900   
4            VIF     FEBI     0.                     0.   0.00        31.820   

  Coût unitaire  Vente 2022  ...  Pareto_BA  Pareto_BB Pareto_BC Pareto_BD  \
0       135.2

In [28]:

#Normalisation des donées
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_to_scale = ['NJR', 'CMJ', 'Vente 2022', 'Vente 2023','Ecoulement']
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

In [29]:

from sklearn.model_selection import train_test_split
features = ['NJR', 'CMJ', 'Vente 2022', 'Vente 2023', 'Ecoulement', 'Importance'] + [col for col in data.columns if col.startswith('Pareto_')]
X=data[features]
y=data['Vente 2024']
# Vérifier à nouveau les valeurs manquantes dans y
print("Vérification des valeurs manquantes dans y après suppression des lignes:")
print(data['Vente 2024'].isna().sum())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Vérification des valeurs manquantes dans y après suppression des lignes:
0


In [30]:
# Entraînement du modèle Random Forest et validation croisée
model = RandomForestRegressor(random_state=42)
cv_scores_rf = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Random Forest Cross-Validated MAE: {-cv_scores_rf.mean()}")

# Entraînement final du modèle
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation finale du modèle
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

# Prédictions complètes pour affichage des résultats finaux
data['Predicted Vente 2024'] = model.predict(X)

# Comparaison avec les valeurs fournies par les responsables
comparison = data[['N°', 'Vente 2024', 'Predicted Vente 2024']]
print(comparison.head())

Random Forest Cross-Validated MAE: 109.74278922331413
Mean Absolute Error: 109.48034439004316
Mean Squared Error: 27688.342777930957
          N°  Vente 2024  Predicted Vente 2024
0  1100936.0    428.9610            208.911270
1     1090.0    343.5755            343.575500
2     1110.0    385.8320            407.795595
3     2113.0    343.5755            343.575500
4     2234.0    558.7590            561.548200
